# Import library

In [61]:
import os
import os, sys
import glob
import re
import datetime
from datetime import datetime, timedelta 
import math, copy, time
from itertools import cycle
from collections import defaultdict

from parse import parse
import numpy as np
import pandas as pd

import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F

# import torchvision
# import torchvision.transforms as transforms

from sklearn.metrics import roc_auc_score, plot_roc_curve
from sklearn import svm


from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import seaborn

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
pd.set_option("display.max_column", 100)

In [63]:
# Util functions
def size_and_ratio(df, col, dropna=True):
    if pd.__version__ > '1.1':
        sr = df.groupby(col, dropna=dropna).size().sort_values(ascending=False)
    elif dropna:
        sr = df.groupby(col).size().sort_values(ascending=False)
    else:  # dropna == False
        sr = df.fillna('nan').groupby(col).size().sort_values(ascending=False)
    sr_ratio = sr.copy() / sum(sr)
    print("Sum :", sum(sr), sr.shape)
    sr.name = 'size'
    sr_ratio.name = 'ratio'
    return pd.concat([sr, sr_ratio], axis=1)

In [67]:
dir_result = "../mc_scripts/model/train_result/"

all_results = dict()
result_dirs = sorted(os.listdir(dir_result))
display(result_dirs)
for train_name in result_dirs:
    
    if not os.path.isdir(os.path.join(dir_result, train_name)):
        continue
    
    if not 'v12' in train_name:
        continue
        
    df_result = pd.read_csv(os.path.join(dir_result, train_name, "result.csv"))
    print(train_name)
    display(df_result)
    all_results[train_name] = df_result

['.DS_Store',
 'all_manual_20230323_014415',
 'owid_manual_20230321_050601',
 'patient+loc+loc-1+loc-2+loc-3+loc-4+loc-5+loc-6+loc-7_manual_20230323_010929',
 'patient+loc_manual_20230323_002327',
 'patient_manual_20230321_050609',
 'patient_manual_20230322_212408',
 'si3_manual_20230321_050605',
 'v10_patient_testmonthvalrandom_20230327_012748',
 'v10_patient_testmonthvalrandom_20230327_012810',
 'v10_patient_testmonthvalrandom_20230327_013715',
 'v10_patient_testmonthvalrandom_20230327_014846',
 'v10_patient_testmonthvalrandom_20230327_022118',
 'v10_patient_testmonthvalrandom_20230327_023343',
 'v10_patient_testmonthvalrandom_20230327_024558',
 'v10_patient_testmonthvalrandom_20230327_050747',
 'v10_patient_testmonthvalrandom_20230327_052517',
 'v10_patient_testmonthvalrandom_20230327_071354',
 'v10_patient_testmonthvalrandom_20230327_084910',
 'v10_patient_testmonthvalrandom_20230327_122900',
 'v11_all_testmonthvalrandom_20230327_180822_2022-03',
 'v11_all_testmonthvalrandom_202303

v12_all_testmonthvalmonth_20230409_232347_2022-03


,Unnamed: 0,0,1,2,3,4,mean,std
0,LR,0.814730,0.807111,0.815724,0.822610,0.824736,0.816982,5.714005e-03
1,XGBoost,0.806862,0.806862,0.806862,0.806862,0.806862,0.806862,1.110223e-16
2,LGBM,0.753670,0.753670,0.753670,0.753670,0.753670,0.753670,0.000000e+00
3,TabNet,0.557580,0.442585,0.355408,0.606015,0.558014,0.503920,8.371262e-02


v12_all_testmonthvalmonth_20230409_232421_2022-04


,Unnamed: 0,0,1,2,3,4,mean,std
0,LR,0.896917,0.895778,0.897901,0.896794,0.897625,0.897003,0.000676
1,XGBoost,0.720680,0.720680,0.720680,0.720680,0.720680,0.720680,0.000000
2,LGBM,0.856246,0.856246,0.856246,0.856246,0.856246,0.856246,0.000000
3,TabNet,0.615321,0.473130,0.371254,0.498219,0.541369,0.499859,0.073384


...

,Unnamed: 0,0,1,2,3,4,mean,std
0,LR,0.835584,0.836962,0.832525,0.836475,0.836525,0.835614,1.468022e-03
1,XGBoost,0.776877,0.776877,0.776877,0.776877,0.776877,0.776877,1.110223e-16
2,LGBM,0.747755,0.747755,0.747755,0.747755,0.747755,0.747755,0.000000e+00
3,TabNet,0.488531,0.590363,0.419127,0.732859,0.781013,0.602379,1.262418e-01


v12_patient_testmonthvalmonth_20230409_232414_2022-04


,Unnamed: 0,0,1,2,3,4,mean,std
0,LR,0.867363,0.867499,0.867827,0.867721,0.868394,0.867761,0.000325
1,XGBoost,0.829915,0.829915,0.829915,0.829915,0.829915,0.829915,0.000000
2,LGBM,0.837509,0.837509,0.837509,0.837509,0.837509,0.837509,0.000000
3,TabNet,0.507875,0.593993,0.413360,0.735524,0.795008,0.609152,0.128669


v12_patient_testmonthvalmonth_20230409_232556_2022-05


,Unnamed: 0,0,1,2,3,4,mean,std
0,LR,0.774129,0.773187,0.772722,0.771687,0.772669,0.772879,0.000724
1,XGBoost,0.782641,0.782641,0.782641,0.782641,0.782641,0.782641,0.000000
2,LGBM,0.781453,0.781453,0.781453,0.781453,0.781453,0.781453,0.000000


# 비교... t-test

In [14]:
v3_12_performance = {
    'RL': [0.8418617324532305, 0.8415487608796515, 0.8416251735299258, 0.8412595824666369, 0.8419745211855217, 0.8416539541029933, 0.0002503429729046297],
    'XGBoost': [0.8397734250529169, 0.8399769252098603, 0.8401759641491977, 0.8404991164501336, 0.8403203245753149, 0.8401491510874848, 0.00025429387104055484],
    'LGBM': [0.8551935665124077, 0.8584086173336836, 0.8574014116665187, 0.8546029699355114, 0.8569555186863247, 0.8565124168268892, 0.0014119193339345117],
    'TabNet': [0.8465492138648236, 0.8526912809963112, 0.8462506554558175, 0.8456920881335273, 0.8485493264247758, 0.847946512975051, 0.002560742725296901]
}

# Print the dictionary
print(v3_12_performance)

v3_1_performance = {
    'RL': [0.838625043318763, 0.8381056968241842, 0.838121731946367, 0.8381724699449158, 0.8386626181573107, 0.8383375120383081, 0.0002513588908400427],
    'XGBoost': [0.8304989459898792, 0.8283381534431956, 0.8300372780691224, 0.8290787606908792, 0.8294619283119947, 0.8294830133010143, 0.0007501863108692136],
    'LGBM': [0.8472786243684077, 0.8440041088205618, 0.8467651414632839, 0.8452445571530047, 0.8453708038239218, 0.8457326471258361, 0.0011673129195661717],
    'TabNet': [0.8299095954171143, 0.8266923994478175, 0.8363573898458905, 0.8356437072436631, 0.8373971598208662, 0.8332000503550703, 0.0041649152103427145]
}

# Print the dictionary
print(v3_1_performance)

{'RL': [0.8418617324532305, 0.8415487608796515, 0.8416251735299258, 0.8412595824666369, 0.8419745211855217, 0.8416539541029933, 0.0002503429729046297], 'XGBoost': [0.8397734250529169, 0.8399769252098603, 0.8401759641491977, 0.8404991164501336, 0.8403203245753149, 0.8401491510874848, 0.00025429387104055484], 'LGBM': [0.8551935665124077, 0.8584086173336836, 0.8574014116665187, 0.8546029699355114, 0.8569555186863247, 0.8565124168268892, 0.0014119193339345117], 'TabNet': [0.8465492138648236, 0.8526912809963112, 0.8462506554558175, 0.8456920881335273, 0.8485493264247758, 0.847946512975051, 0.002560742725296901]}
{'RL': [0.838625043318763, 0.8381056968241842, 0.838121731946367, 0.8381724699449158, 0.8386626181573107, 0.8383375120383081, 0.0002513588908400427], 'XGBoost': [0.8304989459898792, 0.8283381534431956, 0.8300372780691224, 0.8290787606908792, 0.8294619283119947, 0.8294830133010143, 0.0007501863108692136], 'LGBM': [0.8472786243684077, 0.8440041088205618, 0.8467651414632839, 0.84524455

In [20]:
for key, values in v3_12_performance.items():

    v3_12_values = values
    v3_1_values = v3_1_performance[key]
    
    # Perform the paired t-test
    stat, p = ttest_rel(v3_12_values, v3_1_values)

    # Print the results
    print(f"{key}, Paired t-test:")
    print(f" - Statistic: {stat:.4f}")
    print(f" - p-value: {p:.4f}")
    print()

RL, Paired t-test:
 - Statistic: 5.9632
 - p-value: 0.0010

XGBoost, Paired t-test:
 - Statistic: 5.5913
 - p-value: 0.0014

LGBM, Paired t-test:
 - Statistic: 5.5061
 - p-value: 0.0015

TabNet, Paired t-test:
 - Statistic: 3.9345
 - p-value: 0.0077



In [3]:
# Define the dictionary of values
RL_values = {0: 0.838625043318763, 1: 0.8381056968241842, 2: 0.838121731946367, 3: 0.8381724699449158, 4: 0.8386626181573107}

# Assign the values to a list variable
RL_list = [RL_values[i] for i in range(len(RL_values))]

# Print the list variable
print(RL_list)

[0.838625043318763, 0.8381056968241842, 0.838121731946367, 0.8381724699449158, 0.8386626181573107]


In [4]:
from scipy.stats import ttest_rel

# Define the RL performance in the first table and the second table as lists
v3_12_RL = [0.8418617324532305, 0.8415487608796515, 0.8416251735299258, 0.8412595824666369, 0.8419745211855217]
v3_1_RL = [0.838625043318763, 0.8381056968241842, 0.838121731946367, 0.8381724699449158, 0.8386626181573107]

# Perform the paired t-test
stat, p = ttest_rel(table1_RL, table2_RL)

# Print the results
print("Paired t-test:")
print(f"Statistic: {stat:.4f}")
print(f"p-value: {p:.4f}")

Paired t-test:
Statistic: 6.9994
p-value: 0.0022
